In [ ]:
#This notebook assumes you already have spiksortingrecording populated. 

In [230]:
%reload_ext autoreload
%autoreload 2

import os
import numpy as np
import datajoint as dj
import spyglass as nd
# ignore datajoint+jupyter async warnings
import warnings
warnings.simplefilter('ignore', category=DeprecationWarning)
warnings.simplefilter('ignore', category=ResourceWarning)

import spyglass.common as sgc

from spyglass.common import (Session, IntervalList,LabMember, LabTeam, Raw, Session, Nwbfile,
                            Electrode)#,SpikeSortingView)
import spyglass.spikesorting as ss


# from spyglass.spikesorting import (SortGroup, 
#                                     SortInterval,
#                                     SpikeSortingPreprocessingParameters,
#                                     SpikeSortingRecording, 
#                                     SpikeSorterParameters,
#                                     SpikeSortingRecordingSelection,
#                                     ArtifactDetectionParameters, ArtifactDetectionSelection,
#                                     ArtifactRemovedIntervalList, ArtifactDetection,
#                                       SpikeSortingSelection, SpikeSorting,CuratedSpikeSortingSelection,
#                                   CuratedSpikeSorting)#,SpikeSortingView)
# # from spyglass.figurl_views import SpikeSortingView

In [231]:
os.environ['SPYGLASS_BASE_DIR'] = "/stelmo/nwb/"
os.environ['SPYGLASS_RECORDING_DIR'] = "/stelmo/nwb/recording"
os.environ['SPYGLASS_SORTING_DIR'] = "/stelmo/nwb/sorting"
os.environ['SPYGLASS_WAVEFORMS_DIR'] = "/stelmo/nwb/waveforms"
os.environ['SPYGLASS_TEMP_DIR'] = "/stelmo/nwb/tmp"
os.environ['KACHERY_DAEMON_HOST'] = "typhoon"
os.environ['KACHERY_DAEMON_PORT'] = "14747"
os.environ['KACHERY_STORAGE_DIR'] = "/stelmo/nwb/kachery-storage"
os.environ['KACHERY_TEMP_DIR'] = "/stelmo/nwb/tmp"
os.environ['FIGURL_CHANNEL'] = "franklab2"
os.environ['DJ_SUPPORT_FILEPATH_MANAGEMENT'] = "TRUE"
dj.config["enable_python_native_blobs"] = True

In [184]:
nwb_file_name =  'senor20201030_.nwb'
sort_group_id = 24

tetrode = False
if tetrode:
    sort_interval_name= 'runs_noPrePostTrialTimes raw data valid times'
else: 
    sort_interval_name= 'sleeps_runs_noPrePostTrialTimes raw data valid times'

# tetrode = True 

In [157]:
#making sure your spikesortingrecording table has an entry. if not, can adapt the commented code below to insert 

(ss.SpikeSortingRecordingSelection() & {'nwb_file_name' : nwb_file_name, 'sort_interval_name' : sort_interval_name, 'sort_group_id':sort_group_id})

#if not yet populated, can implement the 
# key = dict()

# key['nwb_file_name'] = nwb_file_name
# key['sort_group_id'] = sort_group_id
# key['sort_interval_name'] = 'r1_r2_r3'
# key['preproc_params_name'] = 'franklab_tetrode_hippocampus'
# key['interval_list_name'] = 'r1_r2_r3'
# key['team_name'] = 'mcoulter section'

# ssr_key1 = key

# SpikeSortingRecordingSelection.insert1(ssr_key1, skip_duplicates=True,)
# SpikeSortingRecording.populate([(SpikeSortingRecordingSelection & {'nwb_file_name' : nwb_file_name, 'sort_interval_name' : sort_interval_name, 'sort_group_id':sort_group_id}).proj()])

nwb_file_name name of the NWB file,sort_group_id identifier for a group of electrodes,sort_interval_name name for this interval,preproc_params_name,team_name,interval_list_name descriptive name of this interval list
senor20201030_.nwb,8,runs_noPrePostTrialTimes raw data valid times,franklab_tetrode_hippocampus,ac_em_xs,runs_noPrePostTrialTimes raw data valid times


# artifact detection 

In [144]:
#did lots of testing before to come up with these threhsolds for this particular day
artifact_params = 'ampl_2000_z_30_prop_075_1ms'

In [113]:
artifact_key

{'nwb_file_name': 'senor20201030_.nwb',
 'sort_group_id': 25,
 'sort_interval_name': 'sleeps_runs_noPrePostTrialTimes raw data valid times',
 'preproc_params_name': 'default',
 'team_name': 'ac_em_xs',
 'artifact_params_name': 'ampl_2000_z_30_prop_075_1ms'}

In [158]:
artifact_key = (ss.SpikeSortingRecordingSelection() &  {'nwb_file_name' : nwb_file_name} & 
                     {'sort_interval_name' : sort_interval_name} & {'sort_group_id' : sort_group_id}).fetch1('KEY')
ss.SpikeSortingRecordingSelection() & artifact_key

nwb_file_name name of the NWB file,sort_group_id identifier for a group of electrodes,sort_interval_name name for this interval,preproc_params_name,team_name,interval_list_name descriptive name of this interval list
senor20201030_.nwb,8,runs_noPrePostTrialTimes raw data valid times,franklab_tetrode_hippocampus,ac_em_xs,runs_noPrePostTrialTimes raw data valid times


In [169]:
#this will popualte the artifact detection table, will print out what it found. 
artifact_key = (ss.SpikeSortingRecordingSelection() &  {'nwb_file_name' : nwb_file_name} & 
                     {'sort_interval_name' : sort_interval_name} & {'sort_group_id' : sort_group_id}).fetch1('KEY')

artifact_key['artifact_params_name'] = artifact_params
artifact_key

# ss.ArtifactDetectionSelection.insert1(artifact_key, skip_duplicates=True)
ss.ArtifactDetectionSelection & artifact_key
# ss.ArtifactDetection.populate([(ss.ArtifactDetectionSelection & artifact_key).proj()])
ss.ArtifactDetection & artifact_key

nwb_file_name name of the NWB file,sort_group_id identifier for a group of electrodes,sort_interval_name name for this interval,preproc_params_name,team_name,artifact_params_name,artifact_times np array of artifact intervals,artifact_removed_valid_times np array of valid no-artifact intervals,artifact_removed_interval_list_name name of the array of no-artifact valid time intervals
senor20201030_.nwb,24,sleeps_runs_noPrePostTrialTimes raw data valid times,default,ac_em_xs,ampl_2000_z_30_prop_075_1ms,=BLOB=,=BLOB=,senor20201030_.nwb_sleeps_runs_noPrePostTrialTimes raw data valid times_24_default_ampl_2000_z_30_prop_075_1ms_artifact_removed_valid_times


In [13]:
#just making sure this downstream table has been filled 
ss.ArtifactRemovedIntervalList() &  artifact_key

artifact_removed_interval_list_name,nwb_file_name name of the NWB file,sort_group_id identifier for a group of electrodes,sort_interval_name name for this interval,preproc_params_name,team_name,artifact_params_name,artifact_removed_valid_times,artifact_times np array of artifact intervals
senor20201030_.nwb_sleeps_runs_noPrePostTrialTimes raw data valid times_24_default_ampl_2000_z_30_prop_075_1ms_artifact_removed_valid_times,senor20201030_.nwb,24,sleeps_runs_noPrePostTrialTimes raw data valid times,default,ac_em_xs,ampl_2000_z_30_prop_075_1ms,=BLOB=,=BLOB=


AttributeError: module 'spyglass.spikesorting.sortingview' has no attribute '__version__'

In [35]:
#just to look at potential sorting parameter options 
(SpikeSorterParameters() &{'sorter':'mountainsort4'}).fetch()

array([('mountainsort4', 'default', {'detect_sign': -1, 'adjacency_radius': -1, 'freq_min': 300, 'freq_max': 6000, 'filter': True, 'whiten': True, 'num_workers': 1, 'clip_size': 50, 'detect_threshold': 3, 'detect_interval': 10}),
       ('mountainsort4', 'franklab_probe_ctx_30KHz', {'detect_sign': -1, 'adjacency_radius': 100, 'freq_min': 300, 'freq_max': 6000, 'filter': False, 'whiten': True, 'num_workers': 1, 'clip_size': 40, 'detect_threshold': 3, 'detect_interval': 10}),
       ('mountainsort4', 'franklab_probe_ctx_30KHz_115rad', {'detect_sign': -1, 'adjacency_radius': 115, 'freq_min': 300, 'freq_max': 6000, 'filter': False, 'whiten': True, 'num_workers': 1, 'clip_size': 40, 'detect_threshold': 3, 'detect_interval': 10}),
       ('mountainsort4', 'franklab_tetrode_30KHz', {'detect_sign': -1, 'adjacency_radius': 100, 'freq_min': 300, 'freq_max': 6000, 'filter': False, 'whiten': True, 'num_workers': 4, 'clip_size': 30, 'detect_threshold': 3, 'detect_interval': 10}),
       ('mountains

# populate SpikeSorting(+Selection) tables

In [171]:
if tetrode: 
    spike_sorting_params_name='franklab_tetrode_hippocampus_30KHz'
else: #probes- geometry of the channels calls for the 115rad but i forgot about that at first
    spike_sorting_params_name='franklab_probe_ctx_30KHz'#_115rad'

#get the intervals to use for the sort
artifact_intervals = (ss.ArtifactDetection() & {'nwb_file_name' : nwb_file_name} 
                          & {'artifact_params_name': artifact_params}
                          & {'sort_interval_name' : sort_interval_name} & {'sort_group_id' : sort_group_id}
                         ).fetch('artifact_removed_interval_list_name')[0]
    
sorting_key = (ss.SpikeSortingRecordingSelection() &  {'nwb_file_name' : nwb_file_name} & 
                 {'sort_interval_name' : sort_interval_name} & {'sort_group_id' : sort_group_id}).fetch1('KEY')    
sorting_key['sorter'] = 'mountainsort4'
sorting_key['sorter_params_name'] = spike_sorting_params_name
sorting_key['artifact_removed_interval_list_name'] = artifact_intervals
#do spike sorting selection table
# ss.SpikeSortingSelection.insert1(sorting_key, skip_duplicates=True)

ss.SpikeSortingSelection & sorting_key

nwb_file_name name of the NWB file,sort_group_id identifier for a group of electrodes,sort_interval_name name for this interval,preproc_params_name,team_name,sorter,sorter_params_name,artifact_removed_interval_list_name,import_path optional path to previous curated sorting output
senor20201030_.nwb,24,sleeps_runs_noPrePostTrialTimes raw data valid times,default,ac_em_xs,mountainsort4,franklab_probe_ctx_30KHz,senor20201030_.nwb_sleeps_runs_noPrePostTrialTimes raw data valid times_24_default_ampl_2000_z_30_prop_075_1ms_artifact_removed_valid_times,


In [120]:
%%time
#do actual spike sorting
ss.SpikeSorting.populate([(ss.SpikeSortingSelection & sorting_key).proj()])

ss.SpikeSorting & sorting_key

Running spike sorting on {'nwb_file_name': 'senor20201030_.nwb', 'sort_group_id': 25, 'sort_interval_name': 'sleeps_runs_noPrePostTrialTimes raw data valid times', 'preproc_params_name': 'default', 'team_name': 'ac_em_xs', 'sorter': 'mountainsort4', 'sorter_params_name': 'franklab_probe_ctx_30KHz_115rad', 'artifact_removed_interval_list_name': 'senor20201030_.nwb_sleeps_runs_noPrePostTrialTimes raw data valid times_25_default_ampl_2000_z_30_prop_075_1ms_artifact_removed_valid_times'}...
Mountainsort4 use the OLD spikeextractors mapped with NewToOldRecording


/home/emonroe/miniconda2/envs/spyglass/lib/python3.8/site-packages/neo/io/nixio.py:57: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  MIN_NIX_VER = Version("1.5.0")
/home/emonroe/miniconda2/envs/spyglass/lib/python3.8/site-packages/neo/io/neomatlabio.py:30: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if version.LooseVersion(scipy.version.version) < '0.12.0':
/home/emonroe/miniconda2/envs/spyglass/lib/python3.8/site-packages/setuptools/_distutils/version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
/home/emonroe/miniconda2/envs/spyglass/lib/python3.8/site-packages/neo/io/nixio.py:57: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  MIN_NIX_VER = Version("1.5.0")
/home/emonroe/miniconda2/envs/spyglass/lib/python3.8/site-packages/neo/io/neomatlabio.py:30: Deprec

Saving sorting results...


/home/emonroe/miniconda2/envs/spyglass/lib/python3.8/site-packages/spikeinterface/core/basesorting.py:119: UserWarning: The registered recording will not be persistent on disk, but only available in memory
  warnings.warn(
/home/emonroe/miniconda2/envs/spyglass/lib/python3.8/tempfile.py:818: ResourceWarning: Implicitly cleaning up <TemporaryDirectory '/stelmo/nwb/tmp/tmpa58xe_oo'>
  _warnings.warn(warn_message, ResourceWarning)


CPU times: user 1h 3min 45s, sys: 2h 11min 6s, total: 3h 14min 51s
Wall time: 4h 3min 39s


nwb_file_name name of the NWB file,sort_group_id identifier for a group of electrodes,sort_interval_name name for this interval,preproc_params_name,team_name,sorter,sorter_params_name,artifact_removed_interval_list_name,sorting_path,"time_of_sort in Unix time, to the nearest second"
senor20201030_.nwb,25,sleeps_runs_noPrePostTrialTimes raw data valid times,default,ac_em_xs,mountainsort4,franklab_probe_ctx_30KHz_115rad,senor20201030_.nwb_sleeps_runs_noPrePostTrialTimes raw data valid times_25_default_ampl_2000_z_30_prop_075_1ms_artifact_removed_valid_times,/stelmo/nwb/sorting/senor20201030_.nwb_sleeps_runs_noPrePostTrialTimes raw data valid times_25_default_9cb59d8b_spikesorting,1656538100


In [161]:
#make sure it popualted
ss.SpikeSorting & sorting_key

nwb_file_name name of the NWB file,sort_group_id identifier for a group of electrodes,sort_interval_name name for this interval,preproc_params_name,team_name,sorter,sorter_params_name,artifact_removed_interval_list_name,sorting_path,"time_of_sort in Unix time, to the nearest second"
senor20201030_.nwb,8,runs_noPrePostTrialTimes raw data valid times,franklab_tetrode_hippocampus,ac_em_xs,mountainsort4,franklab_tetrode_hippocampus_30KHz,senor20201030_.nwb_runs_noPrePostTrialTimes raw data valid times_8_franklab_tetrode_hippocampus_ampl_2000_z_30_prop_075_1ms_artifact_removed_valid_times,/stelmo/nwb/sorting/senor20201030_.nwb_runs_noPrePostTrialTimes raw data valid times_8_franklab_tetrode_hippocampus_796b6ea9_spikesorting,1655319578


# Curation

In [172]:
#grt a mySQL error here even though it adds the entry to the table, whats going on there? 
curation_key = sorting_key.copy()
curation_key.update({'curation_id' : 0})

# ss.Curation.insert_curation(curation_key)

ss.Curation() & curation_key

curation_id a number correponding to the index of this curation,nwb_file_name name of the NWB file,sort_group_id identifier for a group of electrodes,sort_interval_name name for this interval,preproc_params_name,team_name,sorter,sorter_params_name,artifact_removed_interval_list_name,parent_curation_id,curation_labels a dictionary of labels for the units,merge_groups a list of merge groups for the units,quality_metrics a list of quality metrics for the units (if available),description optional description for this curated sort,"time_of_creation in Unix time, to the nearest second"
0,senor20201030_.nwb,24,sleeps_runs_noPrePostTrialTimes raw data valid times,default,ac_em_xs,mountainsort4,franklab_probe_ctx_30KHz,senor20201030_.nwb_sleeps_runs_noPrePostTrialTimes raw data valid times_24_default_ampl_2000_z_30_prop_075_1ms_artifact_removed_valid_times,-1,=BLOB=,=BLOB=,=BLOB=,,1656006827


## waveforms

In [173]:
#popualte waveforms table, using whitened (other options clusterless, unwhitened) 
waveform_key_whitened = curation_key.copy()
waveform_key_whitened.update({'waveform_params_name': 'default_whitened'})
# ss.WaveformSelection.insert1(waveform_key_whitened, skip_duplicates=True)
# ss.Waveforms.populate([(ss.WaveformSelection & waveform_key_whitened).proj()])
ss.Waveforms() & waveform_key_whitened




curation_id a number correponding to the index of this curation,nwb_file_name name of the NWB file,sort_group_id identifier for a group of electrodes,sort_interval_name name for this interval,preproc_params_name,team_name,sorter,sorter_params_name,artifact_removed_interval_list_name,waveform_params_name name of waveform extraction parameters,waveform_extractor_path,analysis_file_name name of the file,waveforms_object_id Object ID for the waveforms in NWB file
0,senor20201030_.nwb,24,sleeps_runs_noPrePostTrialTimes raw data valid times,default,ac_em_xs,mountainsort4,franklab_probe_ctx_30KHz,senor20201030_.nwb_sleeps_runs_noPrePostTrialTimes raw data valid times_24_default_ampl_2000_z_30_prop_075_1ms_artifact_removed_valid_times,default_whitened,/stelmo/nwb/waveforms/senor20201030_.nwb_6f844137_0_default_whitened_waveforms,senor20201030_9FAX6MQEZ5.nwb,34ca6fb8-c323-4450-8185-521bc4f4cf14


## Metrics

In [128]:
#useful to pick which metrics you want to use. 
#currently am using my ownn ,with some of jens, but adding peak offset (and peak channel soon)
#very current- i had to comment out the excpetion about doing peak offset with whitened waveforms from the source code (spikesorting.spikesorting_curaiton)

(ss.MetricParameters()).fetch()


array([('DPG_just_peak_offset', {'peak_offset': {'peak_sign': 'neg'}}),
       ('DPG_no_peak_offset', {'snr': {'peak_sign': 'neg', 'num_chunks_per_segment': 20, 'chunk_size': 10000, 'seed': 0}, 'isi_violation': {'isi_threshold_ms': 1.5, 'min_isi_ms': 0.0}, 'nn_isolation': {'max_spikes_for_nn': 1000, 'n_neighbors': 5, 'n_components': 7, 'radius_um': 100, 'seed': 0}, 'nn_noise_overlap': {'max_spikes_for_nn': 1000, 'n_neighbors': 5, 'n_components': 7, 'radius_um': 100, 'seed': 0}}),
       ('DPG_peak_offset', {'snr': {'peak_sign': 'neg', 'num_chunks_per_segment': 20, 'chunk_size': 10000, 'seed': 0}, 'isi_violation': {'isi_threshold_ms': 1.5, 'min_isi_ms': 0.0}, 'nn_isolation': {'max_spikes_for_nn': 1000, 'n_neighbors': 5, 'n_components': 7, 'radius_um': 100, 'seed': 0}, 'nn_noise_overlap': {'max_spikes_for_nn': 1000, 'n_neighbors': 5, 'n_components': 7, 'radius_um': 100, 'seed': 0}, 'peak_offset': {'peak_sign': 'neg'}}),
       ('DPG_test_05_06_22', {'snr': {'peak_sign': 'neg', 'num_chunk

In [130]:
# EM_metrics = ('EM_peak_offset_peak_chan_min_spikes', {'snr': {'peak_sign': 'neg', 'num_chunks_per_segment': 20, 'chunk_size': 10000, 'seed': 0}, 'isi_violation': {'isi_threshold_ms': 1.5, 'min_isi_ms': 0.0}, 'nn_isolation': {'max_spikes_for_nn': 1000, 'n_neighbors': 5, 'n_components': 7, 'radius_um': 100, 'seed': 0, 'min_spikes_for_nn': 10}, 'nn_noise_overlap': {'max_spikes_for_nn': 1000, 'n_neighbors': 5, 'n_components': 7, 'radius_um': 100, 'seed': 0, 'min_spikes_for_nn': 10}, 'peak_offset': {'peak_sign': 'neg'}, 'peak_channel': {'peak_sign': 'neg'}})
# ss.MetricParameters().insert1(EM_metrics)

In [174]:
# metrics_params_whitened = 'JG_DG_no_peak_offset_min_spikes'
metrics_params_whitened = 'EM_peak_offset_peak_chan_min_spikes'
metrics_key_whitened = waveform_key_whitened.copy()
metrics_key_whitened['metric_params_name'] =  metrics_params_whitened

ss.MetricSelection.insert1(metrics_key_whitened, skip_duplicates=True)
ss.QualityMetrics.populate([(ss.MetricSelection & metrics_key_whitened).proj()])


Computed all metrics: {'snr': {1: 3.095255184041835, 2: 3.1196099384592006, 3: 7.279089856404802, 4: 5.199349897432001, 5: 13.066384899347245, 6: 3.2665962248368112, 7: 3.650265264876104, 8: 25.551856854132726, 9: 9.734040706336277, 10: 12.167550882920345, 11: 4.1364781412281415, 12: 11.030608376608377, 13: 19.303564659064662, 14: 11.030608376608377, 15: 44.12243350643351, 16: 30.33417303567304, 17: 9.651782329532331, 18: 9.651782329532331, 19: 2.433510176584069, 20: 5.515304188304189, 21: 1.2167550882920346, 22: 3.3672123059738412, 23: 4.547539377146998, 24: 7.858478962881714, 25: 3.3679195555207344, 26: 3.488968278761777, 27: 7.408524902225274, 28: 3.704262451112637, 29: 6.173770751854395, 30: 13.582295654079669, 31: 17.286558105192306, 32: 20.990820556304943, 33: 5.035459285453077, 34: 3.7765944640898073, 35: 1.4819944351605252, 36: 11.329783392269421, 37: 6.294324106816346, 38: 15.106377856359229, 39: 8.812053749542883, 40: 6.294324106816346, 41: 5.093659388546594, 42: 5.0936593885

/home/emonroe/miniconda2/envs/spyglass/lib/python3.8/site-packages/h5py/_hl/dataset.py:541: DeprecationWarning: Passing None into shape arguments as an alias for () is deprecated.
  arr = numpy.ndarray(selection.mshape, dtype=new_dtype)
/home/emonroe/miniconda2/envs/spyglass/lib/python3.8/site-packages/hdmf/spec/namespace.py:532: UserWarning: Ignoring cached namespace 'hdmf-common' version 1.5.0 because version 1.5.1 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/home/emonroe/miniconda2/envs/spyglass/lib/python3.8/site-packages/hdmf/spec/namespace.py:532: UserWarning: Ignoring cached namespace 'hdmf-experimental' version 0.1.0 because version 0.2.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."


Writing new NWB file senor20201030_YTQE9IMOE9.nwb


/home/emonroe/miniconda2/envs/spyglass/lib/python3.8/site-packages/h5py/_hl/dataset.py:541: DeprecationWarning: Passing None into shape arguments as an alias for () is deprecated.
  arr = numpy.ndarray(selection.mshape, dtype=new_dtype)


Adding metric snr : {1: 3.095255184041835, 2: 3.1196099384592006, 3: 7.279089856404802, 4: 5.199349897432001, 5: 13.066384899347245, 6: 3.2665962248368112, 7: 3.650265264876104, 8: 25.551856854132726, 9: 9.734040706336277, 10: 12.167550882920345, 11: 4.1364781412281415, 12: 11.030608376608377, 13: 19.303564659064662, 14: 11.030608376608377, 15: 44.12243350643351, 16: 30.33417303567304, 17: 9.651782329532331, 18: 9.651782329532331, 19: 2.433510176584069, 20: 5.515304188304189, 21: 1.2167550882920346, 22: 3.3672123059738412, 23: 4.547539377146998, 24: 7.858478962881714, 25: 3.3679195555207344, 26: 3.488968278761777, 27: 7.408524902225274, 28: 3.704262451112637, 29: 6.173770751854395, 30: 13.582295654079669, 31: 17.286558105192306, 32: 20.990820556304943, 33: 5.035459285453077, 34: 3.7765944640898073, 35: 1.4819944351605252, 36: 11.329783392269421, 37: 6.294324106816346, 38: 15.106377856359229, 39: 8.812053749542883, 40: 6.294324106816346, 41: 5.093659388546594, 42: 5.093659388546594, 43:

/home/emonroe/miniconda2/envs/spyglass/lib/python3.8/site-packages/datajoint/hash.py:39: ResourceWarning: unclosed file <_io.BufferedReader name='/stelmo/nwb/analysis/senor20201030_YTQE9IMOE9.nwb'>
  return uuid_from_stream(Path(filepath).open("rb"), init_string=init_string)
/home/emonroe/miniconda2/envs/spyglass/lib/python3.8/site-packages/datajoint/external.py:276: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if check_hash:


In [132]:
# ss.QualityMetrics() & {'nwb_file_name':nwb_file_name,'sort_group_id': 24}
#making sure it popualted
(ss.QualityMetrics() & metrics_key_whitened)#.fetch()



curation_id a number correponding to the index of this curation,nwb_file_name name of the NWB file,sort_group_id identifier for a group of electrodes,sort_interval_name name for this interval,preproc_params_name,team_name,sorter,sorter_params_name,artifact_removed_interval_list_name,waveform_params_name name of waveform extraction parameters,metric_params_name,quality_metrics_path,analysis_file_name name of the file,object_id Object ID for the metrics in NWB file
0,senor20201030_.nwb,25,sleeps_runs_noPrePostTrialTimes raw data valid times,default,ac_em_xs,mountainsort4,franklab_probe_ctx_30KHz_115rad,senor20201030_.nwb_sleeps_runs_noPrePostTrialTimes raw data valid times_25_default_ampl_2000_z_30_prop_075_1ms_artifact_removed_valid_times,default_whitened,EM_peak_offset_peak_chan_min_spikes,/stelmo/nwb/waveforms/senor20201030_.nwb_0a5c795f_0_default_whitened_waveforms_qm.json,senor20201030_IN8FQ87UKX.nwb,9e6be1bd-b730-4d0a-a061-b238e3b12304


In [71]:
#inserting my own autocurationparams- structure is useful 
            
# (ss.AutomaticCurationParameters() & {'auto_curation_params_name':'EM_probe_autocuration_params'}).fetch()
# EM_Autoc = ({'auto_curation_params_name' : 'EM_peak_o,
#             'merge_params' : {},
#             'label_params' : {'nn_noise_overlap': ['>', 0.1, ['noise', 'reject']],
#                                       'isi_violation': ['>', 0.01, ['noise', 'reject']],
#                                       'snr': ['<', 3, ['noise', 'reject']],
#                                       'nn_isolation': ['<', 0.95, ['noise', 'reject']],
#                                       'peak_offset': ['>', 2, ['noise', 'reject']]}})
# ss.AutomaticCurationParameters().insert1(EM_autoc)

## Autocuration 

In [175]:
#do first round of autocuration using the whitened waveforms. 
#then grab the key from this to update for second round. 

if tetrode: 
    # auto_curation_params_name_whitened ='mike_noise_isi_isolation'
    auto_curation_params_name_whitened ='EM_tet_autocuration_params_test2'
else:
    # auto_curation_params_name_whitened = 'EM_probe_autocuration_params'
    auto_curation_params_name_whitened ='EM_probe_autocuration_params_test2'
    
autocuration_key_whitened = metrics_key_whitened.copy()
autocuration_key_whitened['auto_curation_params_name'] = auto_curation_params_name_whitened

ss.AutomaticCurationSelection.insert1(autocuration_key_whitened, skip_duplicates=True)
ss.AutomaticCuration.populate([(ss.AutomaticCurationSelection & autocuration_key_whitened).proj()])

auto_curation_id = (ss.AutomaticCuration & autocuration_key_whitened).fetch1('auto_curation_key')
auto_curation_out_key = (ss.Curation & auto_curation_id).fetch1("KEY")


In [176]:
auto_curation_id2 = (ss.AutomaticCuration & autocuration_key_whitened).fetch1('auto_curation_key')
key2  = (ss.Curation & auto_curation_id2).fetch1("KEY")
ss.SortingviewWorkspaceSelection.insert1(key2, skip_duplicates=True)
ss.SortingviewWorkspace.populate(key2)
ss.SortingviewWorkspace().url(key2)

Creating efficient recording: storing link in kachery...
Done creating efficient recording.
Adding recording: R-c23584c683de
Adding sorting: S-28cdf0c52074
Setting unit metrics for sorting S-28cdf0c52074
{1: ['noise', 'reject', 'noise', 'reject', 'noise', 'reject', 'noise', 'reject', 'noise', 'reject', 'noise', 'reject', 'noise', 'reject', 'noise', 'reject', 'noise', 'reject', 'noise', 'reject', 'noise', 'reject', 'noise', 'reject', 'noise', 'reject', 'noise', 'reject', 'noise', 'reject', 'noise', 'reject', 'noise', 'reject', 'noise', 'reject', 'noise', 'reject', 'noise', 'reject', 'noise', 'reject', 'noise', 'reject', 'noise', 'reject', 'noise', 'reject', 'noise', 'reject', 'noise', 'reject', 'noise', 'reject', 'noise', 'reject', 'noise', 'reject', 'noise', 'reject', 'noise', 'reject', 'noise', 'reject', 'noise', 'reject', 'noise', 'reject', 'noise', 'reject', 'noise', 'reject', 'noise', 'reject', 'noise', 'reject', 'noise', 'reject', 'noise', 'reject', 'noise', 'reject', 'noise', 're

'https://figurl.org/f?v=gs://figurl/spikesortingview-1&d=f4bf06ba3bbb4f62b4d52d461c3e263575d8093e&channel=franklab2&label=senor20201030_.nwb_sleeps_runs_noPrePostTrialTimes%20raw%20data%20valid%20times_24_default'

In [167]:
ss.SortingviewWorkspace().url(key2)

Preparing spikesortingview data


'https://figurl.org/f?v=gs://figurl/spikesortingview-1&d=a5d02abc2e93a1fc6465418f2d5c7f70960be0af&channel=franklab2&label=senor20201030_.nwb_runs_noPrePostTrialTimes%20raw%20data%20valid%20times_8_franklab_tetrode_hippocampus'

## Second set of waveforms, metrics, autocuration- this WAS needed before, when we thought we needed to do whitened and then a second round with unwhitened. maybe not the case for now, but this is the structure if you want to do 2 consecutive rounds of autocuration

In [64]:
waveform_params_not_whitened='default_not_whitened'

waveforms_not_whitened = auto_curation_out_key.copy()
waveforms_not_whitened.update({'waveform_params_name': waveform_params_not_whitened})

ss.WaveformSelection.insert1(waveforms_not_whitened, skip_duplicates=True)
ss.Waveforms.populate([(ss.WaveformSelection & waveforms_not_whitened).proj()])
ss.Waveforms() & waveforms_not_whitened

Extracting waveforms...
Setting 'return_scaled' to False


/home/emonroe/miniconda2/envs/spyglass/lib/python3.8/multiprocessing/popen_fork.py:70: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x7f6ae833b520>
  self.pid = os.fork()
/home/emonroe/miniconda2/envs/spyglass/lib/python3.8/multiprocessing/popen_fork.py:70: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x7f6a168b4f40>
  self.pid = os.fork()
/home/emonroe/miniconda2/envs/spyglass/lib/python3.8/multiprocessing/popen_fork.py:70: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x7f6ae833b520>
  self.pid = os.fork()
/home/emonroe/miniconda2/envs/spyglass/lib/python3.8/multiprocessing/popen_fork.py:70: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x7f6a168b4f40>
  self.pid = os.fork()
/home/emonroe/miniconda2/envs/spyglass/lib/python3.8/multiprocessing/popen_fork.py:70: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x7f6ae833b520>
  self.pid = os.fork()
/home/emonroe/miniconda2/envs/spyglass/lib/python3.8/multiprocessing/popen_fork.

Writing new NWB file senor20201030_MST46VZSPA.nwb


/home/emonroe/miniconda2/envs/spyglass/lib/python3.8/site-packages/h5py/_hl/dataset.py:541: DeprecationWarning: Passing None into shape arguments as an alias for () is deprecated.
  arr = numpy.ndarray(selection.mshape, dtype=new_dtype)
/home/emonroe/miniconda2/envs/spyglass/lib/python3.8/site-packages/hdmf/build/objectmapper.py:256: DtypeConversionWarning: Spec 'Units/spike_times': Value with data type int64 is being converted to data type float64 as specified.
  warnings.warn(full_warning_msg, DtypeConversionWarning)
/home/emonroe/miniconda2/envs/spyglass/lib/python3.8/site-packages/datajoint/hash.py:39: ResourceWarning: unclosed file <_io.BufferedReader name='/stelmo/nwb/analysis/senor20201030_MST46VZSPA.nwb'>
  return uuid_from_stream(Path(filepath).open("rb"), init_string=init_string)
/home/emonroe/miniconda2/envs/spyglass/lib/python3.8/site-packages/datajoint/external.py:276: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this w

curation_id a number correponding to the index of this curation,nwb_file_name name of the NWB file,sort_group_id identifier for a group of electrodes,sort_interval_name name for this interval,preproc_params_name,team_name,sorter,sorter_params_name,artifact_removed_interval_list_name,waveform_params_name name of waveform extraction parameters,waveform_extractor_path,analysis_file_name name of the file,waveforms_object_id Object ID for the waveforms in NWB file
2,senor20201030_.nwb,8,runs_noPrePostTrialTimes raw data valid times,franklab_tetrode_hippocampus,ac_em_xs,mountainsort4,franklab_tetrode_hippocampus_30KHz,senor20201030_.nwb_runs_noPrePostTrialTimes raw data valid times_8_franklab_tetrode_hippocampus_ampl_2000_z_30_prop_075_1ms_artifact_removed_valid_times,default_not_whitened,/stelmo/nwb/waveforms/senor20201030_.nwb_1b9d46c8_2_default_not_whitened_waveforms,senor20201030_MST46VZSPA.nwb,0cee9721-e282-49ae-9921-6f530ecfd046


In [65]:
metrics_params_not_whitened = 'EM_peak_offset_peak_chan'

metrics_key_not_whitened = waveforms_not_whitened.copy()
metrics_key_not_whitened['metric_params_name'] =  metrics_params_not_whitened

ss.MetricSelection.insert1(metrics_key_not_whitened, skip_duplicates=True)
ss.QualityMetrics.populate([(ss.MetricSelection & metrics_key_not_whitened).proj()])

Computed all metrics: {'peak_channel': {1: 32, 2: 32, 3: 32, 4: 32, 5: 33, 6: 33, 7: 33, 8: 33, 9: 33, 10: 34, 11: 33, 12: 34, 13: 35, 14: 35, 15: 35, 16: 35, 17: 35, 18: 35}, 'peak_offset': {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0}}


/home/emonroe/miniconda2/envs/spyglass/lib/python3.8/site-packages/h5py/_hl/dataset.py:541: DeprecationWarning: Passing None into shape arguments as an alias for () is deprecated.
  arr = numpy.ndarray(selection.mshape, dtype=new_dtype)
/home/emonroe/miniconda2/envs/spyglass/lib/python3.8/site-packages/hdmf/spec/namespace.py:532: UserWarning: Ignoring cached namespace 'hdmf-common' version 1.5.0 because version 1.5.1 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/home/emonroe/miniconda2/envs/spyglass/lib/python3.8/site-packages/hdmf/spec/namespace.py:532: UserWarning: Ignoring cached namespace 'hdmf-experimental' version 0.1.0 because version 0.2.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."


Writing new NWB file senor20201030_MBDE6CEXES.nwb


/home/emonroe/miniconda2/envs/spyglass/lib/python3.8/site-packages/h5py/_hl/dataset.py:541: DeprecationWarning: Passing None into shape arguments as an alias for () is deprecated.
  arr = numpy.ndarray(selection.mshape, dtype=new_dtype)


Adding metric peak_channel : {1: 32, 2: 32, 3: 32, 4: 32, 5: 33, 6: 33, 7: 33, 8: 33, 9: 33, 10: 34, 11: 33, 12: 34, 13: 35, 14: 35, 15: 35, 16: 35, 17: 35, 18: 35}
Adding metric peak_offset : {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0}


/home/emonroe/miniconda2/envs/spyglass/lib/python3.8/site-packages/datajoint/hash.py:39: ResourceWarning: unclosed file <_io.BufferedReader name='/stelmo/nwb/analysis/senor20201030_MBDE6CEXES.nwb'>
  return uuid_from_stream(Path(filepath).open("rb"), init_string=init_string)
/home/emonroe/miniconda2/envs/spyglass/lib/python3.8/site-packages/datajoint/external.py:276: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if check_hash:


In [61]:
ss.QualityMetrics & metrics_key_not_whitened

curation_id a number correponding to the index of this curation,nwb_file_name name of the NWB file,sort_group_id identifier for a group of electrodes,sort_interval_name name for this interval,preproc_params_name,team_name,sorter,sorter_params_name,artifact_removed_interval_list_name,waveform_params_name name of waveform extraction parameters,metric_params_name,quality_metrics_path,analysis_file_name name of the file,object_id Object ID for the metrics in NWB file
1,senor20201030_.nwb,24,sleeps_runs_noPrePostTrialTimes raw data valid times,default,ac_em_xs,mountainsort4,franklab_probe_ctx_30KHz,senor20201030_.nwb_sleeps_runs_noPrePostTrialTimes raw data valid times_24_default_ampl_2000_z_30_prop_075_1ms_artifact_removed_valid_times,default_not_whitened,EM_peak_offset_peak_chan,/stelmo/nwb/waveforms/senor20201030_.nwb_ce74b82e_1_default_not_whitened_waveforms_qm.json,senor20201030_2RZCH90B64.nwb,9ece9708-c465-4964-bf52-b8ae94466460


In [ ]:
# 'auto_curation_params_name' : 'EM_peak_offset_pos'

In [73]:
auto_curation_params_name_not_whitened = 'EM_peak_offset_pos'

autocuration_key_not_whitened = metrics_key_not_whitened.copy()
autocuration_key_not_whitened['auto_curation_params_name'] = auto_curation_params_name_not_whitened

ss.AutomaticCurationSelection.insert1(autocuration_key_not_whitened, skip_duplicates=True)
ss.AutomaticCuration.populate([(ss.AutomaticCurationSelection & autocuration_key_not_whitened).proj()])

In [74]:
ss.AutomaticCuration & autocuration_key_not_whitened

curation_id a number correponding to the index of this curation,nwb_file_name name of the NWB file,sort_group_id identifier for a group of electrodes,sort_interval_name name for this interval,preproc_params_name,team_name,sorter,sorter_params_name,artifact_removed_interval_list_name,waveform_params_name name of waveform extraction parameters,metric_params_name,auto_curation_params_name name of this parameter set,auto_curation_key the key to the curation inserted by make
2,senor20201030_.nwb,8,runs_noPrePostTrialTimes raw data valid times,franklab_tetrode_hippocampus,ac_em_xs,mountainsort4,franklab_tetrode_hippocampus_30KHz,senor20201030_.nwb_runs_noPrePostTrialTimes raw data valid times_8_franklab_tetrode_hippocampus_ampl_2000_z_30_prop_075_1ms_artifact_removed_valid_times,default_not_whitened,EM_peak_offset_peak_chan,EM_peak_offset_pos,=BLOB=


In [77]:
ss.WaveformParameters().fetch()

array([('default_clusterless', {'ms_before': 0.5, 'ms_after': 0.5, 'max_spikes_per_unit': None, 'n_jobs': 5, 'total_memory': '5G', 'whiten': False}),
       ('default_not_whitened', {'ms_before': 0.5, 'ms_after': 0.5, 'max_spikes_per_unit': 5000, 'n_jobs': 5, 'total_memory': '5G', 'whiten': False}),
       ('default_whitened', {'ms_before': 0.5, 'ms_after': 0.5, 'max_spikes_per_unit': 5000, 'n_jobs': 5, 'total_memory': '5G', 'whiten': True})],
      dtype=[('waveform_params_name', 'O'), ('waveform_params', 'O')])

In [ ]:
auto_curation_id2 = (ss.AutomaticCuration & autocuration_key_not_whitened).fetch1('auto_curation_key')
key2  = (ss.Curation & auto_curation_id2).fetch1("KEY")
ss.SortingviewWorkspaceSelection.insert1(key2, skip_duplicates=True)
ss.SortingviewWorkspace.populate(key2)
ss.SortingviewWorkspace().url(key2)

In [67]:
ss.SortingviewWorkspace()& key2

curation_id a number correponding to the index of this curation,nwb_file_name name of the NWB file,sort_group_id identifier for a group of electrodes,sort_interval_name name for this interval,preproc_params_name,team_name,sorter,sorter_params_name,artifact_removed_interval_list_name,workspace_uri,sortingview_recording_id,sortingview_sorting_id,channel the name of the kachery channel for data sharing
2,senor20201030_.nwb,24,sleeps_runs_noPrePostTrialTimes raw data valid times,default,ac_em_xs,mountainsort4,franklab_probe_ctx_30KHz,senor20201030_.nwb_sleeps_runs_noPrePostTrialTimes raw data valid times_24_default_ampl_2000_z_30_prop_075_1ms_artifact_removed_valid_times,workspace://30023bd4f41a70e516ea62978635f11c2cc05b398335cc25f9a9614cebf4d81f?label=senor20201030_.nwb_sleeps_runs_noPrePostTrialTimes raw data valid times_24_default,R-931ab86fd65a,S-2e7d405cd836,franklab2


In [69]:
ss.AutomaticCuration & autocuration_key_not_whitened

curation_id a number correponding to the index of this curation,nwb_file_name name of the NWB file,sort_group_id identifier for a group of electrodes,sort_interval_name name for this interval,preproc_params_name,team_name,sorter,sorter_params_name,artifact_removed_interval_list_name,waveform_params_name name of waveform extraction parameters,metric_params_name,auto_curation_params_name name of this parameter set,auto_curation_key the key to the curation inserted by make
1,senor20201030_.nwb,24,sleeps_runs_noPrePostTrialTimes raw data valid times,default,ac_em_xs,mountainsort4,franklab_probe_ctx_30KHz,senor20201030_.nwb_sleeps_runs_noPrePostTrialTimes raw data valid times_24_default_ampl_2000_z_30_prop_075_1ms_artifact_removed_valid_times,default_not_whitened,EM_peak_offset_peak_chan,EM_peak_offset,=BLOB=


In [71]:
ss.AutomaticCurationParameters.fetch()

array([('default', {}, {'nn_noise_overlap': ['>', 0.1, ['noise', 'reject']]}),
       ('EM_peak_offset', {}, {'peak_offset': ['<', -2, ['noise', 'reject']]}),
       ('EM_probe_autocuration_params', {}, {'nn_noise_overlap': ['>', 0.03, ['noise', 'reject']], 'isi_violation': ['>', 0.0025, ['noise', 'reject']], 'snr': ['<', 3, ['noise', 'reject']], 'nn_isolation': ['<', 0.95, ['noise', 'reject']]}),
       ('JG_DG_AutoCuration_params', {}, {'nn_noise_overlap': ['>', 0.03, ['noise', 'reject']], 'isi_violation': ['>', 0.0025, ['noise', 'reject']]}),
       ('mike_noise_isi_isolation', {}, {'nn_noise_overlap': ['>', 0.1, ['noise', 'reject']], 'isi_violation': ['>', 0.01, ['noise', 'reject']], 'nn_isolation': ['<', 0.95, ['noise', 'reject']]}),
       ('mike_noise_isi_isolation_09', {}, {'nn_noise_overlap': ['>', 0.1, ['noise', 'reject']], 'isi_violation': ['>', 0.01, ['noise', 'reject']], 'nn_isolation': ['<', 0.9, ['noise', 'reject']]}),
       ('none', {}, {})],
      dtype=[('auto_curati

## Manual curation

In [116]:
(ss.SortingviewWorkspace & autocuration_key).fetch1('workspace_uri')

DataJointError: fetch1 should only return one tuple. 0 tuples found

### Can add your own labels to the URL from above 

In [117]:
#if you add any labels, can run this after
manual_curation_key = ss.SortingviewWorkspace().insert_manual_curation(key, 'manually curated')

{'curation_id': 1, 'nwb_file_name': 'senor20201030_.nwb', 'sort_group_id': 31, 'sort_interval_name': 'sleeps_runs_noPrePostTrialTimes raw data valid times', 'preproc_params_name': 'default', 'team_name': 'ac_em_xs', 'sorter': 'mountainsort4', 'sorter_params_name': 'franklab_probe_ctx_30KHz', 'artifact_removed_interval_list_name': 'senor20201030_.nwb_sleeps_runs_noPrePostTrialTimes raw data valid times_31_default_ampl_2000_z_30_prop_075_1ms_artifact_removed_valid_times'}


In [ ]:
(ss.Curation() & {'nwb_file_name': 'senor20201030_.nwb','sort_group_id':31}).fetch('curation_labels')
                                                                        

# only when things are pretty final, ready to add into CuratedSpikeSorting, can access spike times etc from this 

In [20]:
#when youre ready to add everything into the final curated tables to look at units, uncomment these 

final_curation_temp_key = (ss.AutomaticCuration & autocuration_key).fetch1('auto_curation_key')
final_curation_key = (ss.Curation & final_curation_temp_key).fetch1("KEY")
final_curation_key
ss.CuratedSpikeSortingSelection.insert1(final_curation_key, skip_duplicates=True)
ss.CuratedSpikeSorting.populate(final_curation_key)
ss.Curation() & final_curation_key

Found 9 accepted units


/home/emonroe/miniconda2/envs/spyglass/lib/python3.8/site-packages/h5py/_hl/dataset.py:541: DeprecationWarning: Passing None into shape arguments as an alias for () is deprecated.
  arr = numpy.ndarray(selection.mshape, dtype=new_dtype)
/home/emonroe/miniconda2/envs/spyglass/lib/python3.8/site-packages/hdmf/spec/namespace.py:532: UserWarning: Ignoring cached namespace 'hdmf-common' version 1.5.0 because version 1.5.1 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/home/emonroe/miniconda2/envs/spyglass/lib/python3.8/site-packages/hdmf/spec/namespace.py:532: UserWarning: Ignoring cached namespace 'hdmf-experimental' version 0.1.0 because version 0.2.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."


Writing new NWB file senor20201030_5CU1BOAZ08.nwb


/home/emonroe/miniconda2/envs/spyglass/lib/python3.8/site-packages/h5py/_hl/dataset.py:541: DeprecationWarning: Passing None into shape arguments as an alias for () is deprecated.
  arr = numpy.ndarray(selection.mshape, dtype=new_dtype)


Adding metric snr : [39.98102692 57.30613858 10.54637818  9.22808091 18.35277761 23.24685164
 33.03499969 23.24685164 23.24685164]
Adding metric isi_violation : [0.         0.00489663 0.00208773 0.00767123 0.         0.
 0.00010224 0.         0.        ]
Adding metric nn_isolation : [0.98931789 0.9867     0.9725     0.9706     0.9923     0.9897
 0.9896     0.99235474 0.9899    ]
Adding metric nn_noise_overlap : [0.0003861  0.         0.0052     0.0195     0.0006     0.0002
 0.         0.00030581 0.0005    ]


/home/emonroe/miniconda2/envs/spyglass/lib/python3.8/site-packages/datajoint/hash.py:39: ResourceWarning: unclosed file <_io.BufferedReader name='/stelmo/nwb/analysis/senor20201030_5CU1BOAZ08.nwb'>
  return uuid_from_stream(Path(filepath).open("rb"), init_string=init_string)
/home/emonroe/miniconda2/envs/spyglass/lib/python3.8/site-packages/datajoint/external.py:276: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if check_hash:


NameError: name 'auto_curation_out_key' is not defined

In [21]:
CuratedSpikeSorting().Unit & final_curation_key

curation_id a number correponding to the index of this curation,nwb_file_name name of the NWB file,sort_group_id identifier for a group of electrodes,sort_interval_name name for this interval,preproc_params_name,team_name,sorter,sorter_params_name,artifact_removed_interval_list_name,unit_id ID for each unit,label optional set of labels for each unit,nn_noise_overlap noise overlap metric for each unit,nn_isolation isolation score metric for each unit,isi_violation ISI violation score for each unit,snr SNR for each unit,firing_rate firing rate,num_spikes total number of spikes,peak_channel channel of maximum amplitude for each unit
1,senor20201030_.nwb,8,runs_noPrePostTrialTimes raw data valid times,franklab_tetrode_hippocampus,ac_em_xs,mountainsort4,franklab_tetrode_hippocampus_30KHz,senor20201030_.nwb_runs_noPrePostTrialTimes raw data valid times_8_franklab_tetrode_hippocampus_ampl_2000_z_30_prop_075_1ms_artifact_removed_valid_times,2,,0.0003861,0.989318,0.0,39.981,-1.0,-1,None
1,senor20201030_.nwb,8,runs_noPrePostTrialTimes raw data valid times,franklab_tetrode_hippocampus,ac_em_xs,mountainsort4,franklab_tetrode_hippocampus_30KHz,senor20201030_.nwb_runs_noPrePostTrialTimes raw data valid times_8_franklab_tetrode_hippocampus_ampl_2000_z_30_prop_075_1ms_artifact_removed_valid_times,4,,0.0,0.9867,0.00489663,57.3061,-1.0,-1,None
1,senor20201030_.nwb,8,runs_noPrePostTrialTimes raw data valid times,franklab_tetrode_hippocampus,ac_em_xs,mountainsort4,franklab_tetrode_hippocampus_30KHz,senor20201030_.nwb_runs_noPrePostTrialTimes raw data valid times_8_franklab_tetrode_hippocampus_ampl_2000_z_30_prop_075_1ms_artifact_removed_valid_times,11,,0.0052,0.9725,0.00208773,10.5464,-1.0,-1,None
1,senor20201030_.nwb,8,runs_noPrePostTrialTimes raw data valid times,franklab_tetrode_hippocampus,ac_em_xs,mountainsort4,franklab_tetrode_hippocampus_30KHz,senor20201030_.nwb_runs_noPrePostTrialTimes raw data valid times_8_franklab_tetrode_hippocampus_ampl_2000_z_30_prop_075_1ms_artifact_removed_valid_times,12,,0.0195,0.9706,0.00767123,9.22808,-1.0,-1,None
1,senor20201030_.nwb,8,runs_noPrePostTrialTimes raw data valid times,franklab_tetrode_hippocampus,ac_em_xs,mountainsort4,franklab_tetrode_hippocampus_30KHz,senor20201030_.nwb_runs_noPrePostTrialTimes raw data valid times_8_franklab_tetrode_hippocampus_ampl_2000_z_30_prop_075_1ms_artifact_removed_valid_times,13,,0.0006,0.9923,0.0,18.3528,-1.0,-1,None
1,senor20201030_.nwb,8,runs_noPrePostTrialTimes raw data valid times,franklab_tetrode_hippocampus,ac_em_xs,mountainsort4,franklab_tetrode_hippocampus_30KHz,senor20201030_.nwb_runs_noPrePostTrialTimes raw data valid times_8_franklab_tetrode_hippocampus_ampl_2000_z_30_prop_075_1ms_artifact_removed_valid_times,15,,0.0002,0.9897,0.0,23.2469,-1.0,-1,None
1,senor20201030_.nwb,8,runs_noPrePostTrialTimes raw data valid times,franklab_tetrode_hippocampus,ac_em_xs,mountainsort4,franklab_tetrode_hippocampus_30KHz,senor20201030_.nwb_runs_noPrePostTrialTimes raw data valid times_8_franklab_tetrode_hippocampus_ampl_2000_z_30_prop_075_1ms_artifact_removed_valid_times,16,,0.0,0.9896,0.000102239,33.035,-1.0,-1,None
1,senor20201030_.nwb,8,runs_noPrePostTrialTimes raw data valid times,franklab_tetrode_hippocampus,ac_em_xs,mountainsort4,franklab_tetrode_hippocampus_30KHz,senor20201030_.nwb_runs_noPrePostTrialTimes raw data valid times_8_franklab_tetrode_hippocampus_ampl_2000_z_30_prop_075_1ms_artifact_removed_valid_times,17,,0.00030581,0.992355,0.0,23.2469,-1.0,-1,None
1,senor20201030_.nwb,8,runs_noPrePostTrialTimes raw data valid times,franklab_tetrode_hippocampus,ac_em_xs,mountainsort4,franklab_tetrode_hippocampus_30KHz,senor20201030_.nwb_runs_noPrePostTrialTimes raw data valid times_8_franklab_tetrode_hippocampus_ampl_2000_z_30_prop_075_1ms_artifact_removed_valid_times,18,,0.0005,0.9899,0.0,23.2469,-1.0,-1,None


In [178]:
#need to test how to get sampling rate and how to get tetrode or probe 

In [250]:
(ss.SpikeSorterParameters & {'sorter':'mountainsort4'}).fetch()

array([('mountainsort4', 'default', {'detect_sign': -1, 'adjacency_radius': -1, 'freq_min': 300, 'freq_max': 6000, 'filter': True, 'whiten': True, 'num_workers': 1, 'clip_size': 50, 'detect_threshold': 3, 'detect_interval': 10}),
       ('mountainsort4', 'franklab_probe_ctx_30KHz', {'detect_sign': -1, 'adjacency_radius': 100, 'freq_min': 300, 'freq_max': 6000, 'filter': False, 'whiten': True, 'num_workers': 1, 'clip_size': 40, 'detect_threshold': 3, 'detect_interval': 10}),
       ('mountainsort4', 'franklab_probe_ctx_30KHz_115rad', {'detect_sign': -1, 'adjacency_radius': 115, 'freq_min': 300, 'freq_max': 6000, 'filter': False, 'whiten': True, 'num_workers': 1, 'clip_size': 40, 'detect_threshold': 3, 'detect_interval': 10}),
       ('mountainsort4', 'franklab_tetrode_30KHz', {'detect_sign': -1, 'adjacency_radius': 100, 'freq_min': 300, 'freq_max': 6000, 'filter': False, 'whiten': True, 'num_workers': 4, 'clip_size': 30, 'detect_threshold': 3, 'detect_interval': 10}),
       ('mountains

In [255]:
(ss.Curation & {'nwb_file_name':nwb_file_name,'sort_group_id':27}).delete()

[2022-07-06 12:29:54,222][INFO]: Deleting 3 rows from `spikesorting_curation`.`__automatic_curation`
[2022-07-06 12:29:54,238][INFO]: Deleting 3 rows from `spikesorting_curation`.`automatic_curation_selection`
[2022-07-06 12:29:54,254][INFO]: Deleting 3 rows from `spikesorting_curation`.`__quality_metrics`
[2022-07-06 12:29:54,273][INFO]: Deleting 3 rows from `spikesorting_curation`.`metric_selection`
[2022-07-06 12:29:54,289][INFO]: Deleting 1 rows from `spikesorting_curation`.`__waveforms`
[2022-07-06 12:29:54,303][INFO]: Deleting 1 rows from `spikesorting_curation`.`waveform_selection`
[2022-07-06 12:29:54,332][INFO]: Deleting 2 rows from `spikesorting_sortingview`.`__sortingview_workspace`
[2022-07-06 12:29:54,343][INFO]: Deleting 3 rows from `spikesorting_sortingview`.`sortingview_workspace_selection`
[2022-07-06 12:29:54,352][INFO]: Deleting 4 rows from `spikesorting_curation`.`curation`


Commit deletes? [yes, No]:  yes


Deletes committed.


4

In [257]:
dj.ERD(Session)[-1:1]

TypeError: unhashable type: 'slice'